### Enrichissement du dataset initial (GouvImmo)

Dans les différents documents consultés à propos des estimations immobilières, il est souvent évoqué
l'enrichissement du dataset par des données complémentaires : liées aux équipements à proximité des biens,
aux nuisances potentielles,... et la plupart du temps l'ajout du prix moyen au m2 dans une zone proche du bien...
Compte tenu de la pauvreté de notre dataset intial, nous allons donc travailler cette piste. 

Dans la suite de ce notebook nous allons chercher à obtenir le prix au m2 moyen par commune et arrondissement
sur des sites différents de GouvImmo. Ce afin d'éviter au maximum toute dépendance avec la variable cible.
Pour cela nous aurons besoin de 2 sites différents et de 2 techniques de récupérations des données :
Webscrapping et copier/coller + Excel...

Ces nouvelles données ainsi constituées seront sauvegardées dans 2 fichiers distincts (avec ou sans encodage complémentaire)

##### 1ère partie dédiée à la création des 2 types de dataset (et fichiers csv associés)

In [ ]:
# Importation des librairies
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
pd.pandas.set_option('display.max_columns',None)
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Lasso, LassoCV

In [ ]:
# Chargement du dataset
df=pd.read_csv("IDF_2019_to_2021_v3.csv")
df.head()

,id_mutation,valeur_fonciere,date_mutation,numero_disposition,nature_mutation,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,id_parcelle,nombre_lots,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude,Appartement,Dépendance,Maison,terrains_a_batir,terrains_autre,prix_m²,section_cadastrale,prix_m²_moy_ca,prix_m²_moy_co,annee,code_annee,type_bien
0,2019-1135463,310000.0,2019-01-03,1.0,Vente,11.0,NaN,RUE DE LA FONTAINE,0650,77220.0,77215,Gretz-Armainvilliers,77,772150000B0473,0,101.0,4.0,855.0,2.740574,48.739669,0,0,1,0,1,3069.31,772150000B,3646.883495,3474.511880,2019,0,Maison
1,2019-1135466,155000.0,2019-01-04,1.0,Vente,4.0,NaN,AV DE LA GARE,0600,77340.0,77373,Pontault-Combault,77,77373000AD0157,2,51.0,2.0,0.0,2.616840,48.805639,1,1,0,0,0,3039.22,77373000AD,3804.018010,3759.720085,2019,0,Appartement
2,2019-1135467,165500.0,2019-01-09,1.0,Vente,45.0,NaN,RUE LOUIS VICTOR DE BROGLIE,1413,77176.0,77445,Savigny-le-Temple,77,77445000YA0444,3,64.0,3.0,12.0,2.565832,48.605740,1,2,0,0,0,2585.94,77445000YA,2821.665116,2610.471778,2019,0,Appartement
3,2019-1135468,269000.0,2019-01-02,1.0,Vente,6.0,NaN,RUE JEAN FERRAT,0492,77310.0,77407,Saint-Fargeau-Ponthierry,77,77407000AY0353,0,84.0,4.0,270.0,2.553743,48.530118,0,0,1,0,2,3202.38,77407000AY,2815.332174,2679.428891,2019,0,Maison
4,2019-1135469,199500.0,2019-01-08,1.0,Vente,3.0,NaN,RUE DE LA PICARDIE,0186,77720.0,77211,Grandpuits-Bailly-Carrois,77,772110170D0147,0,109.0,5.0,1036.0,2.993174,48.576719,0,0,1,0,2,1830.28,772110170D,2360.692500,2020.605714,2019,0,Maison


In [ ]:
#Retrait des variables prix/m² car créées via la variables cible pour le data viz, et ne devant pas êtres incluses dans le modèle
df=df.drop(["prix_m²_moy_ca", "prix_m²_moy_co", "prix_m²"], axis=1)


In [ ]:
# Suppression des lignes avec latitude / longitude à Nan =========
print('Nbr de lignes sans lat/lon Nan :', len(df[(df['latitude'].notna() & df['longitude'].notna())]))

df = df[(df['latitude'].notna() & df['longitude'].notna())]

Nbr de lignes sans lat/lon Nan : 363029


##### Utilisation d'un autre site Web (efficity.com) pour récupérer des prix m2 moyen (par commune)

In [ ]:
# Chargement des libairies nécessaires à BeautifulSoup
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

In [ ]:
# Accès à la page du site web ciblé (le fichier robots.txt ne faisait pas référence à cette page...)
site_rt = "https://www.efficity.com/prix-immobilier-m2/r_ile-de-france_11/"
req = Request(url=site_rt)
# Lecture de la page
page_rt = urlopen(req).read()

In [ ]:
# Acquisition du code HTML
soup = BeautifulSoup(page_rt, 'html.parser')

# Utilisation de SelectorGadget pour cibler les champs localisations et départements visés
list_loc = soup.select(".squaremeter-locations-list span")
list_loc

[<span data-clamp="1">Alfortville (94)</span>,
 <span data-clamp="1">Antony (92)</span>,
 <span data-clamp="1">Argenteuil (95)</span>,
 <span data-clamp="1">Asnières-sur-Seine (92)</span>,
 <span data-clamp="1">Aubervilliers (93)</span>,
 <span data-clamp="1">Aulnay-sous-Bois (93)</span>,
 <span data-clamp="1">Bagneux (92)</span>,
 <span data-clamp="1">Bagnolet (93)</span>,
 <span data-clamp="1">Bobigny (93)</span>,
 <span data-clamp="1">Bondy (93)</span>,
 <span data-clamp="1">Boulogne-Billancourt (92)</span>,
 <span data-clamp="1">Cergy (95)</span>,
 <span data-clamp="1">Champigny-sur-Marne (94)</span>,
 <span data-clamp="1">Châtillon (92)</span>,
 <span data-clamp="1">Chelles (77)</span>,
 <span data-clamp="1">Choisy-le-Roi (94)</span>,
 <span data-clamp="1">Clamart (92)</span>,
 <span data-clamp="1">Clichy (92)</span>,
 <span data-clamp="1">Colombes (92)</span>,
 <span data-clamp="1">Conflans-Sainte-Honorine (78)</span>,
 <span data-clamp="1">Corbeil-Essonnes (91)</span>,
 <span da

In [ ]:
# Extraction de la liste des localisations et des départements
import re
dept = re.compile(r'\(([0-9]{2})\)')

localisation = []
departement = []
for element in list_loc[0:-23]:
    localisation.append(element.text.split(" (")[0])
    departement.append(dept.findall(element.text)[0])
    
print(localisation)
print(departement)

['Alfortville', 'Antony', 'Argenteuil', 'Asnières-sur-Seine', 'Aubervilliers', 'Aulnay-sous-Bois', 'Bagneux', 'Bagnolet', 'Bobigny', 'Bondy', 'Boulogne-Billancourt', 'Cergy', 'Champigny-sur-Marne', 'Châtillon', 'Chelles', 'Choisy-le-Roi', 'Clamart', 'Clichy', 'Colombes', 'Conflans-Sainte-Honorine', 'Corbeil-Essonnes', 'Courbevoie', 'Créteil', 'Drancy', 'Épinay-sur-Seine', 'Évry', 'Fontenay-sous-Bois', 'Gagny', 'Garges-lès-Gonesse', 'Gennevilliers', 'Issy-les-Moulineaux', 'Ivry-sur-Seine', 'La Courneuve', 'Le Blanc-Mesnil', 'Le Perreux-sur-Marne', 'Levallois-Perret', 'Livry-Gargan', 'Maisons-Alfort', 'Mantes-la-Jolie', 'Massy', 'Meaux', 'Melun', 'Meudon', 'Montigny-le-Bretonneux', 'Montreuil', 'Montrouge', 'Nanterre', 'Neuilly-sur-Marne', 'Neuilly-sur-Seine', 'Noisy-le-Grand', 'Noisy-le-Sec', 'Pantin', 'Paris', 'Poissy', 'Pontault-Combault', 'Puteaux', 'Rosny-sous-Bois', 'Rueil-Malmaison', 'Saint-Denis', 'Saint-Germain-en-Laye', 'Saint-Maur-des-Fossés', 'Saint-Ouen-sur-Seine', 'Sainte-G

In [ ]:
# utilisation de SelectorGadget pour cible le champ prix au m2 visé
list_prix = soup.select(".squaremeter-locations-list small")
list_prix

[<small>5710€/m<sup>2</sup></small>,
 <small>5990€/m<sup>2</sup></small>,
 <small>3880€/m<sup>2</sup></small>,
 <small>6860€/m<sup>2</sup></small>,
 <small>4500€/m<sup>2</sup></small>,
 <small>3720€/m<sup>2</sup></small>,
 <small>5340€/m<sup>2</sup></small>,
 <small>5870€/m<sup>2</sup></small>,
 <small>3870€/m<sup>2</sup></small>,
 <small>3540€/m<sup>2</sup></small>,
 <small>9040€/m<sup>2</sup></small>,
 <small>3180€/m<sup>2</sup></small>,
 <small>4850€/m<sup>2</sup></small>,
 <small>6040€/m<sup>2</sup></small>,
 <small>3890€/m<sup>2</sup></small>,
 <small>3950€/m<sup>2</sup></small>,
 <small>6610€/m<sup>2</sup></small>,
 <small>7470€/m<sup>2</sup></small>,
 <small>5600€/m<sup>2</sup></small>,
 <small>3950€/m<sup>2</sup></small>,
 <small>2690€/m<sup>2</sup></small>,
 <small>7290€/m<sup>2</sup></small>,
 <small>4120€/m<sup>2</sup></small>,
 <small>3720€/m<sup>2</sup></small>,
 <small>3200€/m<sup>2</sup></small>,
 <small>2390€/m<sup>2</sup></small>,
 <small>5960€/m<sup>2</sup></small>,
 

In [ ]:
# Extraction de la liste des prix au m2
prix = []
for element in list_prix[0:-23]:
    prix.append(element.text.split('€')[0])

print(prix)

['5710', '5990', '3880', '6860', '4500', '3720', '5340', '5870', '3870', '3540', '9040', '3180', '4850', '6040', '3890', '3950', '6610', '7470', '5600', '3950', '2690', '7290', '4120', '3720', '3200', '2390', '5960', '3940', '2690', '4910', '8750', '5420', '2960', '3850', '6490', '9830', '3390', '6000', '2670', '4720', '3020', '2930', '6520', '4160', '6530', '7880', '5680', '3700', '11280', '4560', '4010', '6340', '10380', '4150', '3860', '7800', '4200', '7010', '4340', '7280', '6860', '6660', '3620', '2790', '4520', '3650', '2850', '3100', '7580', '3260']


In [ ]:
# Creation du DataFrame de synthèse des données récupérées
prix_efficity = pd.DataFrame(list(zip(localisation,departement,prix)), columns=["Localisation","Departement","Prix_moy"])

prix_efficity.head()


,Localisation,Departement,Prix_moy
0,Alfortville,94,5710
1,Antony,92,5990
2,Argenteuil,95,3880
3,Asnières-sur-Seine,92,6860
4,Aubervilliers,93,4500


In [ ]:
# Sauvegarde de ces premières données dans un fichier CSV 
prix_efficity.to_csv("efficity.csv", index=False)

In [ ]:
# Test de relecture du fichier précédemment sauvegardé
prix_efficity = pd.read_csv("efficity.csv")
prix_efficity

,Localisation,Departement,Prix_moy
0,Alfortville,94,5710
1,Antony,92,5990
2,Argenteuil,95,3880
3,Asnières-sur-Seine,92,6860
4,Aubervilliers,93,4500
...,...,...,...
65,Savigny-sur-Orge,91,3650
66,Sevran,93,2850
67,Stains,93,3100
68,Suresnes,92,7580


##### Cas particulier de Paris. Dans le fichier ainsi récupéré, nous ne disposons que d'un seul prix moyen pour tout Paris.

In [ ]:
prix_efficity[prix_efficity['Departement']==75]

,Localisation,Departement,Prix_moy
52,Paris,75,10380


In [ ]:
df[df['code_departement']==75]

,id_mutation,valeur_fonciere,date_mutation,numero_disposition,nature_mutation,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,id_parcelle,nombre_lots,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude,Appartement,Dépendance,Maison,terrains_a_batir,terrains_autre,section_cadastrale,annee,code_annee,type_bien
131546,2019-1499564,1196000.0,2019-01-04,1.0,Vente,17.0,NaN,RUE DUPHOT,2999,75001.0,75101,Paris 1er Arrondissement,75,75101000BC0014,2,112.0,3.0,0.0,2.325288,48.868416,1,0,0,0,0,75101000BC,2019,0,Appartement
131547,2019-1499565,1570490.0,2019-01-03,1.0,Vente,13.0,NaN,RUE DE THORIGNY,9298,75003.0,75103,Paris 3e Arrondissement,75,75103000AL0015,3,104.0,3.0,0.0,2.363076,48.860305,1,1,0,0,0,75103000AL,2019,0,Appartement
131548,2019-1499567,5400.0,2019-01-03,1.0,Vente,79.0,NaN,RUE DES GRAVILLIERS,4302,75003.0,75103,Paris 3e Arrondissement,75,75103000AV0022,1,21.0,1.0,0.0,2.353479,48.864674,1,0,0,0,0,75103000AV,2019,0,Appartement
131549,2019-1499568,230000.0,2019-01-10,1.0,Vente,4.0,NaN,RUE BLONDEL,1021,75003.0,75103,Paris 3e Arrondissement,75,75103000AB0130,1,26.0,1.0,0.0,2.354961,48.868615,1,0,0,0,0,75103000AB,2019,0,Appartement
131550,2019-1499570,955750.0,2019-01-05,1.0,Vente,82.0,NaN,BD MALESHERBES,5951,75008.0,75108,Paris 8e Arrondissement,75,75108000CG0021,2,80.0,3.0,0.0,2.314393,48.880118,1,0,0,0,0,75108000CG,2019,0,Appartement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364457,2021-510054,563620.0,2021-03-03,1.0,Vente,17.0,NaN,RUE DES TOURNELLES,9382,75004.0,75104,Paris 4e Arrondissement,75,75104000AO0075,2,33.0,1.0,0.0,2.366987,48.854745,1,0,0,0,0,75104000AO,2021,2,Appartement
364458,2021-510056,190000.0,2021-03-17,1.0,Vente,47.0,NaN,RUE D ABOUKIR,0040,75002.0,75102,Paris 2e Arrondissement,75,75102000AJ0044,1,15.0,1.0,0.0,2.345464,48.867430,1,0,0,0,0,75102000AJ,2021,2,Appartement
364459,2021-510057,230000.0,2021-03-29,1.0,Vente,119.0,NaN,RUE D ABOUKIR,0040,75002.0,75102,Paris 2e Arrondissement,75,75102000AO0165,3,15.0,1.0,0.0,2.350035,48.868906,1,0,0,0,0,75102000AO,2021,2,Appartement
364460,2021-510059,383000.0,2021-03-12,1.0,Vente,8.0,NaN,RUE DES TOURNELLES,9382,75004.0,75104,Paris 4e Arrondissement,75,75104000AO0117,2,30.0,2.0,0.0,2.367712,48.854200,1,0,0,0,0,75104000AO,2021,2,Appartement


##### Pour compléter le dataset par un prix au m2 moyen par arrondissement, nous avons copier/coller dans excel
##### un tableau fourni par le site "notairesdugrandparis.fr" au format PDF.

In [ ]:
# Prix issus de la base Biens stockés dans un 2ème DataFrame
prix_arr_paris = pd.read_excel("prix_arr_paris.xlsx")
prix_arr_paris

,Localisation,Departement,Prix_moy
0,Paris 1er Arrondissement,75,13560
1,Paris 2e Arrondissement,75,12020
2,Paris 3e Arrondissement,75,12780
3,Paris 4e Arrondissement,75,12770
4,Paris 5e Arrondissement,75,12340
5,Paris 6e Arrondissement,75,13980
6,Paris 7e Arrondissement,75,13300
7,Paris 8e Arrondissement,75,11810
8,Paris 9e Arrondissement,75,10800
9,Paris 10e Arrondissement,75,10050


In [ ]:
# Concaténation des prix issus des 2 bases... et des 2 DataFrames de prix...
prix_moy = pd.concat([prix_efficity, prix_arr_paris],axis=0).reset_index().drop(columns='index')
prix_moy

,Localisation,Departement,Prix_moy
0,Alfortville,94,5710
1,Antony,92,5990
2,Argenteuil,95,3880
3,Asnières-sur-Seine,92,6860
4,Aubervilliers,93,4500
...,...,...,...
85,Paris 16e Arrondissement,75,10810
86,Paris 17e Arrondissement,75,10450
87,Paris 18e Arrondissement,75,9750
88,Paris 19e Arrondissement,75,8940


In [ ]:
# Vérification du cas de Paris
prix_moy[prix_moy['Departement']==75]

,Localisation,Departement,Prix_moy
52,Paris,75,10380
70,Paris 1er Arrondissement,75,13560
71,Paris 2e Arrondissement,75,12020
72,Paris 3e Arrondissement,75,12780
73,Paris 4e Arrondissement,75,12770
74,Paris 5e Arrondissement,75,12340
75,Paris 6e Arrondissement,75,13980
76,Paris 7e Arrondissement,75,13300
77,Paris 8e Arrondissement,75,11810
78,Paris 9e Arrondissement,75,10800


In [ ]:
# Fusion du DataFrame initial avec le DataFrame des prix au m2, par clé : code dept et nom commune
# On ne conserve que les lignes complètes.

df_plus = df.merge(right=prix_moy, how='inner', left_on=['code_departement','nom_commune'], 
                   right_on=['Departement','Localisation'])

df_plus.drop(columns=['Departement','Localisation'],inplace=True)

In [ ]:
df_plus

,id_mutation,valeur_fonciere,date_mutation,numero_disposition,nature_mutation,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,id_parcelle,nombre_lots,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude,Appartement,Dépendance,Maison,terrains_a_batir,terrains_autre,section_cadastrale,annee,code_annee,type_bien,Prix_moy
0,2019-1135466,155000.0,2019-01-04,1.0,Vente,4.0,NaN,AV DE LA GARE,0600,77340.0,77373,Pontault-Combault,77,77373000AD0157,2,51.0,2.0,0.0,2.616840,48.805639,1,1,0,0,0,77373000AD,2019,0,Appartement,3860
1,2019-1135471,287000.0,2019-01-03,1.0,Vente,9.0,NaN,RUE LAFAYETTE,0726,77340.0,77373,Pontault-Combault,77,77373000AH0274,0,89.0,4.0,166.0,2.615606,48.794421,0,0,1,0,0,77373000AH,2019,0,Maison,3860
2,2019-1135496,125000.0,2019-01-02,1.0,Vente,100.0,NaN,AV DE LA REPUBLIQUE,1080,77340.0,77373,Pontault-Combault,77,77373000AL0063,3,28.0,1.0,0.0,2.606803,48.802302,1,2,0,0,0,77373000AL,2019,0,Appartement,3860
3,2019-1135540,355000.0,2019-01-03,1.0,Vente,10.0,NaN,RUE DE LA PIERRE ROLLET,0960,77340.0,77373,Pontault-Combault,77,77373000AR0285,0,97.0,5.0,572.0,2.610761,48.790031,0,0,1,0,1,77373000AR,2019,0,Maison,3860
4,2019-1135542,99000.0,2019-01-04,1.0,Vente,69.0,NaN,AV DU GENERAL DE GAULLE,0606,77340.0,77373,Pontault-Combault,77,77373000AD0020,2,17.0,1.0,0.0,2.613044,48.802085,1,1,0,0,0,77373000AD,2019,0,Appartement,3860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177993,2021-510048,808745.0,2021-03-22,1.0,Vente,55.0,NaN,RUE MONTMARTRE,6513,75002.0,75102,Paris 2e Arrondissement,75,75102000AK0086,2,58.0,2.0,0.0,2.344134,48.865842,1,0,0,0,0,75102000AK,2021,2,Appartement,12020
177994,2021-510049,323050.0,2021-03-12,1.0,Vente,193.0,NaN,RUE SAINT-DENIS,8525,75002.0,75102,Paris 2e Arrondissement,75,75102000AM0044,1,25.0,2.0,0.0,2.350421,48.866281,1,0,0,0,0,75102000AM,2021,2,Appartement,12020
177995,2021-510056,190000.0,2021-03-17,1.0,Vente,47.0,NaN,RUE D ABOUKIR,0040,75002.0,75102,Paris 2e Arrondissement,75,75102000AJ0044,1,15.0,1.0,0.0,2.345464,48.867430,1,0,0,0,0,75102000AJ,2021,2,Appartement,12020
177996,2021-510057,230000.0,2021-03-29,1.0,Vente,119.0,NaN,RUE D ABOUKIR,0040,75002.0,75102,Paris 2e Arrondissement,75,75102000AO0165,3,15.0,1.0,0.0,2.350035,48.868906,1,0,0,0,0,75102000AO,2021,2,Appartement,12020


##### Résultat : le dataset est à peu de chose près, réduit de moitié...

In [ ]:
# Sélection des variables numériques de df pour le modèle de régression:
num_df=df_plus.select_dtypes(include=['int','float', "uint8"]).dropna(axis = 0, how = 'any')
num_df

,valeur_fonciere,numero_disposition,adresse_numero,code_postal,code_commune,code_departement,nombre_lots,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude,Appartement,Dépendance,Maison,terrains_a_batir,terrains_autre,annee,code_annee,Prix_moy
0,155000.0,1.0,4.0,77340.0,77373,77,2,51.0,2.0,0.0,2.616840,48.805639,1,1,0,0,0,2019,0,3860
1,287000.0,1.0,9.0,77340.0,77373,77,0,89.0,4.0,166.0,2.615606,48.794421,0,0,1,0,0,2019,0,3860
2,125000.0,1.0,100.0,77340.0,77373,77,3,28.0,1.0,0.0,2.606803,48.802302,1,2,0,0,0,2019,0,3860
3,355000.0,1.0,10.0,77340.0,77373,77,0,97.0,5.0,572.0,2.610761,48.790031,0,0,1,0,1,2019,0,3860
4,99000.0,1.0,69.0,77340.0,77373,77,2,17.0,1.0,0.0,2.613044,48.802085,1,1,0,0,0,2019,0,3860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177993,808745.0,1.0,55.0,75002.0,75102,75,2,58.0,2.0,0.0,2.344134,48.865842,1,0,0,0,0,2021,2,12020
177994,323050.0,1.0,193.0,75002.0,75102,75,1,25.0,2.0,0.0,2.350421,48.866281,1,0,0,0,0,2021,2,12020
177995,190000.0,1.0,47.0,75002.0,75102,75,1,15.0,1.0,0.0,2.345464,48.867430,1,0,0,0,0,2021,2,12020
177996,230000.0,1.0,119.0,75002.0,75102,75,3,15.0,1.0,0.0,2.350035,48.868906,1,0,0,0,0,2021,2,12020


In [ ]:
# Suppression des variables numériques inutiles (avec les coefs les plus bas dans l'analyse initile)
num_df=num_df.drop(["numero_disposition", "adresse_numero", "nombre_lots",
#                    "Appartement", "terrains_a_batir",
#                    "Maison"
#                    "code_commune",
#                    "annee",
#                    "latitude","longitude",
#                    "code_departement",                   
                    "code_annee", "code_postal"], axis=1)

num_df

,valeur_fonciere,code_commune,code_departement,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude,Appartement,Dépendance,Maison,terrains_a_batir,terrains_autre,annee,Prix_moy
0,155000.0,77373,77,51.0,2.0,0.0,2.616840,48.805639,1,1,0,0,0,2019,3860
1,287000.0,77373,77,89.0,4.0,166.0,2.615606,48.794421,0,0,1,0,0,2019,3860
2,125000.0,77373,77,28.0,1.0,0.0,2.606803,48.802302,1,2,0,0,0,2019,3860
3,355000.0,77373,77,97.0,5.0,572.0,2.610761,48.790031,0,0,1,0,1,2019,3860
4,99000.0,77373,77,17.0,1.0,0.0,2.613044,48.802085,1,1,0,0,0,2019,3860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177993,808745.0,75102,75,58.0,2.0,0.0,2.344134,48.865842,1,0,0,0,0,2021,12020
177994,323050.0,75102,75,25.0,2.0,0.0,2.350421,48.866281,1,0,0,0,0,2021,12020
177995,190000.0,75102,75,15.0,1.0,0.0,2.345464,48.867430,1,0,0,0,0,2021,12020
177996,230000.0,75102,75,15.0,1.0,0.0,2.350035,48.868906,1,0,0,0,0,2021,12020


In [ ]:
num_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176301 entries, 0 to 177997
Data columns (total 15 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   valeur_fonciere            176301 non-null  float64
 1   code_commune               176301 non-null  int64  
 2   code_departement           176301 non-null  int64  
 3   surface_reelle_bati        176301 non-null  float64
 4   nombre_pieces_principales  176301 non-null  float64
 5   surface_terrain            176301 non-null  float64
 6   longitude                  176301 non-null  float64
 7   latitude                   176301 non-null  float64
 8   Appartement                176301 non-null  int64  
 9   Dépendance                 176301 non-null  int64  
 10  Maison                     176301 non-null  int64  
 11  terrains_a_batir           176301 non-null  int64  
 12  terrains_autre             176301 non-null  int64  
 13  annee                      17

In [ ]:
# Sauvegarde du nouveau dataset pour test des différents modèles par la suite
num_df.to_csv("prix_autre_site.csv", index=False)

In [ ]:
# Variante AVEC Encodage de annee et de code departement
num_df['annee'] = num_df['annee'].astype('object')
num_df['code_departement'] = num_df['code_departement'].astype('object')

num_df = num_df.join(pd.get_dummies(num_df[['annee','code_departement']], prefix=['an','dep']).astype('int'))
num_df.drop(columns=['annee','code_departement'],inplace=True)

num_df

,valeur_fonciere,code_commune,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude,Appartement,Dépendance,Maison,terrains_a_batir,terrains_autre,Prix_moy,an_2019,an_2020,an_2021,dep_75,dep_77,dep_78,dep_91,dep_92,dep_93,dep_94,dep_95
0,155000.0,77373,51.0,2.0,0.0,2.616840,48.805639,1,1,0,0,0,3860,1,0,0,0,1,0,0,0,0,0,0
1,287000.0,77373,89.0,4.0,166.0,2.615606,48.794421,0,0,1,0,0,3860,1,0,0,0,1,0,0,0,0,0,0
2,125000.0,77373,28.0,1.0,0.0,2.606803,48.802302,1,2,0,0,0,3860,1,0,0,0,1,0,0,0,0,0,0
3,355000.0,77373,97.0,5.0,572.0,2.610761,48.790031,0,0,1,0,1,3860,1,0,0,0,1,0,0,0,0,0,0
4,99000.0,77373,17.0,1.0,0.0,2.613044,48.802085,1,1,0,0,0,3860,1,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177993,808745.0,75102,58.0,2.0,0.0,2.344134,48.865842,1,0,0,0,0,12020,0,0,1,1,0,0,0,0,0,0,0
177994,323050.0,75102,25.0,2.0,0.0,2.350421,48.866281,1,0,0,0,0,12020,0,0,1,1,0,0,0,0,0,0,0
177995,190000.0,75102,15.0,1.0,0.0,2.345464,48.867430,1,0,0,0,0,12020,0,0,1,1,0,0,0,0,0,0,0
177996,230000.0,75102,15.0,1.0,0.0,2.350035,48.868906,1,0,0,0,0,12020,0,0,1,1,0,0,0,0,0,0,0


In [ ]:
# Sauvegarde du nouveau dataset version avec encodage pour test des différents modèles par la suite
num_df.to_csv("prix_autre_site_enc.csv", index=False)

### ----------------------------------------------------------------------------------------------------------------------------------

### Test d'autres modèles de régression

Devant les difficultés rencontrées à faire progresser les résultats des 4 modèles choisis initialement
(régression linéraire, Lasso, Ridge, ElasticNet)
nous avons décidé de tester d'autres modèles évoqués dans la litérature des simulations immobilières...

Ce notebook a pour objectif de comparer ces différents modèles de régression en utilisant la version du dataset
complété de la donnée prix au m2 issu d'autres sites web. Version sans encodage (puis avec).

In [ ]:
# Importation des librairies
#%matplotlib inline
#import matplotlib.pyplot as plt
#import seaborn as sns
#import pandas as pd
#import numpy as np
#pd.pandas.set_option('display.max_columns',None)
#import warnings
#warnings.filterwarnings('ignore')
#from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate, train_test_split
#from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error
#from sklearn.preprocessing import PolynomialFeatures, StandardScaler
#from sklearn.linear_model import Lasso, LassoCV

In [ ]:
# Chargement du dataset spécifique 
df = pd.read_csv("prix_autre_site.csv")
df.head()

,valeur_fonciere,code_commune,code_departement,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude,Appartement,Dépendance,Maison,terrains_a_batir,terrains_autre,annee,Prix_moy
0,155000.0,77373,77,51.0,2.0,0.0,2.616840,48.805639,1,1,0,0,0,2019,3860
1,287000.0,77373,77,89.0,4.0,166.0,2.615606,48.794421,0,0,1,0,0,2019,3860
2,125000.0,77373,77,28.0,1.0,0.0,2.606803,48.802302,1,2,0,0,0,2019,3860
3,355000.0,77373,77,97.0,5.0,572.0,2.610761,48.790031,0,0,1,0,1,2019,3860
4,99000.0,77373,77,17.0,1.0,0.0,2.613044,48.802085,1,1,0,0,0,2019,3860


In [ ]:
# Séparation de la variable cible:
target = df["valeur_fonciere"]
data = df.drop("valeur_fonciere", axis=1)

In [ ]:
# Création des ensembles d'entrainement et de test:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=256)
X_train.head()

,code_commune,code_departement,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude,Appartement,Dépendance,Maison,terrains_a_batir,terrains_autre,annee,Prix_moy
165343,75120,75,56.0,2.0,0.0,2.399386,48.852148,1,0,0,0,0,2019,9330
54224,93055,93,34.0,2.0,0.0,2.410998,48.894735,1,0,0,0,0,2020,6340
165214,75120,75,55.0,3.0,0.0,2.401005,48.868143,1,0,0,0,0,2019,9330
84392,94068,94,30.0,2.0,0.0,2.475332,48.800633,1,0,0,0,0,2020,6860
126045,75112,75,72.0,4.0,0.0,2.376648,48.845978,1,0,0,0,0,2019,9690


In [ ]:
# Standardisation des variables:
scaler = StandardScaler()
X_train=pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test=pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)
X_train.head()

,code_commune,code_departement,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude,Appartement,Dépendance,Maison,terrains_a_batir,terrains_autre,annee,Prix_moy
0,-1.058418,-1.055097,-0.099252,-0.572055,-0.308187,0.362256,-0.125146,0.395027,-0.672245,-0.388139,-0.063813,-0.103787,-0.96645,0.601000
1,0.997482,0.999321,-0.758248,-0.572055,-0.308187,0.450537,0.490981,0.395027,-0.672245,-0.388139,-0.063813,-0.103787,0.39580,-0.402431
2,-1.058418,-1.055097,-0.129206,0.187418,-0.308187,0.374564,0.106261,0.395027,-0.672245,-0.388139,-0.063813,-0.103787,-0.96645,0.601000
3,1.113603,1.113455,-0.878065,-0.572055,-0.308187,0.939642,-0.870440,0.395027,-0.672245,-0.388139,-0.063813,-0.103787,0.39580,-0.227922
4,-1.059335,-1.055097,0.380017,0.946891,-0.308187,0.189388,-0.214411,0.395027,-0.672245,-0.388139,-0.063813,-0.103787,-0.96645,0.721814


### Modèle Lasso simple comme modèle de référence (sortie de l'itération 1)

In [ ]:
# Modèle lasso sur le dataset 
lasso_r = Lasso().fit(X_train, y_train)
print(lasso_r.score(X_train, y_train))
print(lasso_r.score(X_test, y_test))

0.7092218769230719
0.7176816609422235


In [ ]:
# Métriques du modèle:
lasso_r_pred_train =lasso_r.predict(X_train)
lasso_r_pred_test = lasso_r.predict(X_test)

print("Score train: ", lasso_r.score(X_train, y_train))
print("Score test: ", lasso_r.score(X_test, y_test))
print("\n")
print("MAE train: ", mean_absolute_error(y_train, lasso_r_pred_train))
print("MAE test: ", mean_absolute_error(y_test, lasso_r_pred_test))
print("\n")
print("MedAE train: ", median_absolute_error(y_train, lasso_r_pred_train))
print("MedAE test: ", median_absolute_error(y_test, lasso_r_pred_test))
print("\n")
print("RMSE train: ", np.sqrt(mean_squared_error(y_train, lasso_r_pred_train)))
print("RMSE test: ", np.sqrt(mean_squared_error(y_test, lasso_r_pred_test)))

Score train:  0.7092218769230719
Score test:  0.7176816609422235


MAE train:  122812.120406699
MAE test:  121848.61149424061


MedAE train:  81301.09389472866
MedAE test:  82126.91049210867


RMSE train:  191901.01636808278
RMSE test:  188873.96199102962


### Rappel des résultats du modèle Lasso avec suppression des var à faible coef. et sans l'ajout du prix au m2

Score train: 0.5802637318733554
Score test: 0.5865002705462182

MAE train: 125236.34966951564
MAE test: 124693.99967441423
MedAE train: 90153.08864134608
MedAE test: 89791.11863763022
RMSE train: 189400.2224220019
RMSE test: 187317.19180273497¶

#### On peut constater un net progrès au niveau des scores, beaucoup moins sur les erreurs...

On peut également constater que le jeu de test répond un tout petit peu mieux au modèle que le jeu d'entrainement (hasard de la séparation des jeux).

##### Dans cette version comparative des modèles de régression, pas d'appel de la fonction PolynomialFeatures

In [ ]:
#Test avec un traitement polynomial:
#poly=PolynomialFeatures(3)

#X_train_poly=poly.fit_transform(X_train)
#X_test_poly=poly.transform(X_test)

In [ ]:
#Modèle lasso sur le dataset avec un preprocessing polynomial
#lasso_r3 = Lasso().fit(X_train_poly, y_train)
#print(lasso_r3.score(X_train_poly, y_train))
#print(lasso_r3.score(X_test_poly, y_test))

In [ ]:
#Métriques du modèle:
#lasso_r3_pred_train =lasso_r3.predict(X_train_poly)
#lasso_r3_pred_test = lasso_r3.predict(X_test_poly)

#print("Score train: ", lasso_r3.score(X_train_poly, y_train))
#print("Score test: ", lasso_r3.score(X_test_poly, y_test))
#print("\n")
#print("MAE train: ", mean_absolute_error(y_train, lasso_r3_pred_train))
#print("MAE test: ", mean_absolute_error(y_test, lasso_r3_pred_test))
#print("\n")
#print("MedAE train: ", median_absolute_error(y_train, lasso_r3_pred_train))
#print("MedAE test: ", median_absolute_error(y_test, lasso_r3_pred_test))
#print("\n")
#print("RMSE train: ", np.sqrt(mean_squared_error(y_train, lasso_r3_pred_train)))
#print("RMSE test: ", np.sqrt(mean_squared_error(y_test, lasso_r3_pred_test)))

### Test des autres modèles

##### Calcul des métriques uniquement lorsque le résultat est encourageant...

#### GradientBoostingRegressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(random_state=256)
gbr.fit(X_train,y_train)

print('Score Train :', gbr.score(X_train, y_train))
print('Score Test :', gbr.score(X_test, y_test))

Score Train : 0.827603325144996
Score Test : 0.8345898575941946


In [ ]:
#Métriques du modèle:
gbr_pred_train = gbr.predict(X_train)
gbr_pred_test = gbr.predict(X_test)

print("Score train: ", gbr.score(X_train, y_train))
print("Score test: ", gbr.score(X_test, y_test))
print("\n")
print("MAE train: ", mean_absolute_error(y_train, gbr_pred_train))
print("MAE test: ", mean_absolute_error(y_test, gbr_pred_test))
print("\n")
print("MedAE train: ", median_absolute_error(y_train, gbr_pred_train))
print("MedAE test: ", median_absolute_error(y_test, gbr_pred_test))
print("\n")
print("RMSE train: ", np.sqrt(mean_squared_error(y_train, gbr_pred_train)))
print("RMSE test: ", np.sqrt(mean_squared_error(y_test, gbr_pred_test)))

Score train:  0.827603325144996
Score test:  0.8345898575941946


MAE train:  79884.31361539804
MAE test:  79164.2908481527


MedAE train:  43967.29999800239
MedAE test:  43998.44752378494


RMSE train:  147761.35089210377
RMSE test:  144571.80829603775


#### Résultats encore en net progrès, cette fois-ci les erreurs se sont également améliorées...

On constate aussi le même phénomène entre jeu de test et jeu d'entrainement...

##### RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(random_state=256)
rfr.fit(X_train,y_train)

print('Score Train :', rfr.score(X_train, y_train))
print('Score Test :', rfr.score(X_test, y_test))

Score Train : 0.9738540582059931
Score Test : 0.8408359208160612


In [ ]:
#Métriques du modèle:
rfr_pred_train = rfr.predict(X_train)
rfr_pred_test = rfr.predict(X_test)

print("Score train: ", rfr.score(X_train, y_train))
print("Score test: ", rfr.score(X_test, y_test))
print("\n")
print("MAE train: ", mean_absolute_error(y_train, rfr_pred_train))
print("MAE test: ", mean_absolute_error(y_test, rfr_pred_test))
print("\n")
print("MedAE train: ", median_absolute_error(y_train, rfr_pred_train))
print("MedAE test: ", median_absolute_error(y_test, rfr_pred_test))
print("\n")
print("RMSE train: ", np.sqrt(mean_squared_error(y_train, rfr_pred_train)))
print("RMSE test: ", np.sqrt(mean_squared_error(y_test, rfr_pred_test)))

Score train:  0.9738540582059931
Score test:  0.8408359208160612


MAE train:  28963.93960638895
MAE test:  73564.99079631451


MedAE train:  14611.76999999999
MedAE test:  37119.46428571429


RMSE train:  57543.80233108005
RMSE test:  141815.94914239107


#### Résultats encourageants, mais beaucoup d'overfitting...

##### MLPRegressor

In [ ]:
from sklearn.neural_network import MLPRegressor
mlpr = MLPRegressor(random_state=256)
mlpr.fit(X_train,y_train)

print('Score Train :', mlpr.score(X_train, y_train))
print('Score Test :', mlpr.score(X_test, y_test))

Score Train : 0.7757768260757378
Score Test : 0.7852083762398419


#### Résutats plutôt corrects

##### AdaBoostRegressor

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
abr = AdaBoostRegressor(random_state=256)
abr.fit(X_train,y_train)

print('Score Train :', abr.score(X_train, y_train))
print('Score Test :', abr.score(X_test, y_test))

Score Train : 0.6780990668820226
Score Test : 0.6801049469196632


#### Résutats moyens

##### DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtr = AdaBoostRegressor(random_state=256)
dtr.fit(X_train,y_train)

print('Score Train :', dtr.score(X_train, y_train))
print('Score Test :', dtr.score(X_test, y_test))

Score Train : 0.6780990668820226
Score Test : 0.6801049469196632


#### Résutats moyens

##### SVR

In [ ]:
from sklearn.svm import SVR
svr = SVR()
svr.fit(X_train,y_train)

print('Score Train :', svr.score(X_train, y_train))
print('Score Test :', svr.score(X_test, y_test))

#### Beaucoup trop long ???  Noyau stoppé...

##### SGDRegressor

In [ ]:
from sklearn.linear_model import SGDRegressor
sgdr = SGDRegressor()
sgdr.fit(X_train,y_train)

print('Score Train :', sgdr.score(X_train, y_train))
print('Score Test :', sgdr.score(X_test, y_test))

Score Train : 0.7087926562940328
Score Test : 0.717509947266316


#### Résutats moyens

##### KNeighborsRegressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor()
knn.fit(X_train,y_train)

print('Score Train :', knn.score(X_train, y_train))
print('Score Test :', knn.score(X_test, y_test))

Score Train : 0.8655422198688427
Score Test : 0.8080132765088779


In [ ]:
#Métriques du modèle:
knn_pred_train = knn.predict(X_train)
knn_pred_test = knn.predict(X_test)

print("Score train: ", knn.score(X_train, y_train))
print("Score test: ", knn.score(X_test, y_test))
print("\n")
print("MAE train: ", mean_absolute_error(y_train, knn_pred_train))
print("MAE test: ", mean_absolute_error(y_test, knn_pred_test))
print("\n")
print("MedAE train: ", median_absolute_error(y_train, knn_pred_train))
print("MedAE test: ", median_absolute_error(y_test, knn_pred_test))
print("\n")
print("RMSE train: ", np.sqrt(mean_squared_error(y_train, knn_pred_train)))
print("RMSE test: ", np.sqrt(mean_squared_error(y_test, knn_pred_test)))

Score train:  0.8655422198688427
Score test:  0.8080132765088779


MAE train:  69091.40096415201
MAE test:  83731.98064320354


MedAE train:  35513.2
MedAE test:  43988.0


RMSE train:  130493.65001515883
RMSE test:  155753.6180617715


#### Résultats encourageants, mais présence d'overfitting (moins marqué qu'avec rfr)...

##### LinearRegression

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train,y_train)

print('Score Train :', lr.score(X_train, y_train))
print('Score Test :', lr.score(X_test, y_test))

Score Train : 0.7097662809393082
Score Test : 0.7179126095412132


#### Résutats moyens

##### Ridge

In [ ]:
from sklearn.linear_model import Ridge
rr = Ridge(random_state=256)
rr.fit(X_train,y_train)

print('Score Train :', rr.score(X_train, y_train))
print('Score Test :', rr.score(X_test, y_test))

Score Train : 0.7097574705443868
Score Test : 0.7179455086596869


#### Résutats moyens

##### ElasticNet

In [ ]:
from sklearn.linear_model import ElasticNet
encv = ElasticNet(random_state=256)
encv.fit(X_train,y_train)

print('Score Train :', encv.score(X_train, y_train))
print('Score Test :', encv.score(X_test, y_test))

Score Train : 0.6192494802712372
Score Test : 0.6201426006486772


#### Résutats assez faibles

### Conclusions : 
### Aux vues des résultats, il semblait effectivement utile de relancer une phase de tests plus large.
### Le modèle GradientBoostingRegressor semble être une meilleure base de départ que les 4 modèles étudiés
### initialement (itération 1) : Regr. linéaire, Lasso, Ridge, ElasticNet...

### ----------------------------------------------------------------------------------------------------------------------------------

### Un autre notebook a été créé pour tester la version du Dataset AVEC encodage de l'année et du code département.
##### nom : Test_models_prix_site_enc.ipynb

### Les résultats sont sensiblement les mêmes, avec les mêmes tendances/comportements constatés, 

##### soit, par exemple pour le GBR (meilleur modèle à ce stade)

#### L'encodage des variables annee en plus du code_departement n'est pas d'un apport significatif.

### ----------------------------------------------------------------------------------------------------------------------------------

### Autre exploration : l'exploitation du prix au m2 par section cadastrale (ald de prix via sites Web)
##### nom du notebook : Test_models_prix_moy_section.ipynb

#### Comme évoqué dans le notebook de création du dataset (Data_prix_moy_section.ipynb), 
#### il est probable que la dépendance entre la valeur foncière et le prix au m2 par section cadastrale influe 
#### sur le résultat.

#### Nous constatons un léger mieux d'environ un pourcent mais toujours pas de quoi réduire les erreurs de façon
#### significative. De nouveau le modèle GBR est le mieux placé :

##### soit, par exemple pour le GBR

#### Comme pour le test précédent, un notebook a été créé avec un dataset disposant de l'encodage des variables
#### année et code département. Là encore pas de progrès significatifs.
##### nom du notebook : Test_models_prix_moy_section_enc.ipynb

### Enfin, à ces 2 notebook s'ajoutent 3 notebook de tests sur 1 type de bien et 1 département
##### noms des notebook : Test_models_prix_moy_appart75.ipynb, Test_models_prix_moy_appart92.ipynb, Test_models_prix_moy_mais78.ipynb

#### On constate alors que les résultats varient d'un cas particulier à l'autre mais en restant tj dans la même zone
#### des 81% +/- 3%. Par conséquent, le découpage du dataset initial ne permet pas non plus d'améliorer les scores.

##### Exemples de scores obtenus avec le modèle GBR :

In [ ]:
appart75
Score Train : 0.8292808964906394
Score Test : 0.818299858013786
appart92    
Score Train : 0.8448470541844345
Score Test : 0.8376297574530408
mais78
Score Train : 0.8234051351316718
Score Test : 0.7890230677453083

### ----------------------------------------------------------------------------------------------------------------------------------

### Conclusions générales :
    
### De cette nouvelle vague de tests de modèles de régression appliqué à GouvImmo, il ressort :
### - que le GradientBoostingRegressor est pratiquement toujours le mieux placé
### - que l'encodage de certaines variables telles que l'année en plus du département, n'apporte pas un mieux significatif
### - que le découpage du dataset par département en plus du type de bien n'est pas une piste efficace non plus